In [ ]:
access = "bO7TjrSeBvicEYPiQO"      # 실제 API KEY로 수정
secret = "JCO0yReFocd9h8q8YRxO9drDVSJYBA2ncw10"     # 실제 API SECRET로 수정

In [17]:
def get_latest_kline(symbol, interval="5", limit=5, category="linear"):
    url = "https://api.bybit.com/v5/market/kline"
    params = {
        "category": category,
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    data = response.json()

    if data["retCode"] != 0:
        print("Kline 데이터 조회 오류:", data["retMsg"])
        return None

    if "list" not in data["result"] or not data["result"]["list"]:
        print("캔들 데이터가 비어있습니다.")
        return None

    kline_list = data["result"]["list"]
    df = pd.DataFrame(kline_list)
    # 문서에 따르면 [startTime, openPrice, highPrice, lowPrice, closePrice, volume, turnover] 순서
    df.columns = ["startTime", "open", "high", "low", "close", "volume", "turnover"]

    # 밀리초 단위로 가정
    df["startTime"] = df["startTime"].astype("int64")
    df["startTime"] = pd.to_datetime(df["startTime"], unit="ms")
    df.set_index("startTime", inplace=True)

    for col in ["open", "high", "low", "close", "volume", "turnover"]:
        df[col] = df[col].astype(float)

    return df

get_latest_kline(symbol, interval="5", limit=15, category="linear")


,open,high,low,close,volume,turnover
startTime,,,,,,
2025-03-16 08:30:00,1919.26,1920.58,1919.07,1920.05,892.26,1.713163e+06
2025-03-16 08:25:00,1918.99,1919.69,1918.50,1919.26,653.18,1.253490e+06
2025-03-16 08:20:00,1919.04,1920.36,1918.41,1918.99,1029.70,1.976466e+06
2025-03-16 08:15:00,1922.05,1922.11,1917.08,1919.04,2098.57,4.027744e+06
2025-03-16 08:10:00,1922.67,1923.00,1920.22,1922.05,2176.57,4.182044e+06
2025-03-16 08:05:00,1924.08,1924.08,1922.66,1922.67,826.92,1.590527e+06
2025-03-16 08:00:00,1925.09,1926.06,1924.08,1924.08,576.28,1.109458e+06
2025-03-16 07:55:00,1925.62,1925.86,1924.12,1925.09,1152.52,2.218442e+06
2025-03-16 07:50:00,1924.57,1926.30,1923.68,1925.62,855.36,1.646313e+06


In [36]:
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import requests
from pybit.unified_trading import HTTP as UnifiedHTTP

# --- LINE Notify 설정 ---
TARGET_URL = 'https://notify-api.line.me/api/notify'
TOKEN = "rlMIJRZSatEVj5MLBSqC0iVVRIM7trYKqVbwizh7gUL"
def send_line_notification(message):
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"message": message}
    response = requests.post(TARGET_URL, headers=headers, data=data)
    return response

def notify(message):
    print(message)
    send_line_notification(message)

# --- Bybit API 설정 ---
# Unified Trading 클라이언트 (거래, 잔고 등)
access = "f"      # 실제 API KEY로 수정
secret = "f"     # 실제 API SECRET로 수정
# testnet 여부는 필요에 따라 True/False로 설정
bybit_session = UnifiedHTTP(api_key=access, api_secret=secret, testnet=False)

# --- Market Data 함수 (v5 Public API 사용) ---
def get_latest_kline(symbol, interval="5", limit=15, category="linear"):
    url = "https://api.bybit.com/v5/market/kline"
    params = {
        "category": category,
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    data = response.json()

    if data["retCode"] != 0:
        print("Kline 데이터 조회 오류:", data["retMsg"])
        return None

    if "list" not in data["result"] or not data["result"]["list"]:
        print("캔들 데이터가 비어있습니다.")
        return None

    kline_list = data["result"]["list"]
    df = pd.DataFrame(kline_list)
    # 문서에 따르면 [startTime, openPrice, highPrice, lowPrice, closePrice, volume, turnover] 순서
    df.columns = ["startTime", "open", "high", "low", "close", "volume", "turnover"]

    # 밀리초 단위로 가정
    df["startTime"] = df["startTime"].astype("int64")
    df["startTime"] = pd.to_datetime(df["startTime"], unit="ms")
    df.set_index("startTime", inplace=True)

    for col in ["open", "high", "low", "close", "volume", "turnover"]:
        df[col] = df[col].astype(float)

    return df

def get_current_price(symbol, category="linear"):
    url = "https://api.bybit.com/v5/market/ticker"
    params = {"symbol": symbol, "category": category}
    response = requests.get(url, params=params)
    if not response.text.strip():
        notify("응답 내용이 비어 있습니다 (get_current_price).")
        return None
    try:
        data = response.json()
    except Exception as e:
        notify(f"JSON 파싱 에러 in get_current_price: {e}")
        return None
    # 결과가 없으면 오류 처리
    ticker_list = data.get("result", {}).get("list", [])
    if data.get("retCode", -1) != 0 or not ticker_list:
        notify("현재가 조회 오류: " + data.get("retMsg", "Unknown error"))
        return None
    try:
        price = float(ticker_list[0]["lastPrice"])
    except Exception as e:
        notify("현재가 변환 오류: " + str(e))
        return None
    return price


def get_balance_usdt():
    data = bybit_session.get_wallet_balance(coin="USDT")
    if data['retCode'] != 0:
        notify("잔고 조회 오류: " + data['retMsg'])
        return None
    result = data.get("result", {})
    # 결과가 리스트 형태로 반환되는 경우
    if "list" in result:
        for item in result["list"]:
            if item.get("coin") == "USDT":
                return float(item.get("wallet_balance", 0))
        notify("USDT 잔고 데이터가 없습니다.")
        return None
    # 혹은 결과가 딕셔너리 형태로 반환되는 경우 (구버전)
    elif "USDT" in result:
        return float(result["USDT"]["wallet_balance"])
    else:
        notify("USDT 잔고 데이터가 없습니다.")
        return None


# 수정된 place_order 함수: 주문 생성 시 "category" 파라미터 추가
def place_order(symbol, side, qty, category="linear"):
    order = bybit_session.create_order(
        symbol=symbol,
        category=category,          # 필수 파라미터 추가
        orderType="Market",
        side=side,
        qty=qty,
        timeInForce="GoodTillCancel"
    )
    if order['retCode'] != 0:
        notify(f"주문 실행 오류: {order['retMsg']}")
    return order

# --- 전처리 함수 (동일) ---
def rolling_minmax_scale(series, window=6):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

# --- Diffusion Model 구성 (동일) ---
class ConditionEncoder(nn.Module):
    def __init__(self, input_dim, lookback, condition_dim):
        super(ConditionEncoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim * lookback, condition_dim),
            nn.ReLU(),
            nn.Linear(condition_dim, condition_dim)
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        x = x.contiguous().view(batch_size, -1)
        return self.fc(x)

class DiffusionClassifier(nn.Module):
    def __init__(self, input_dim, lookback, condition_dim=128, num_timesteps=100, hidden_dim=128):
        super(DiffusionClassifier, self).__init__()
        self.num_timesteps = num_timesteps
        betas = torch.linspace(1e-4, 0.02, num_timesteps)
        alphas = 1 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        self.register_buffer('betas', betas)
        self.register_buffer('alphas', alphas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        self.condition_encoder = ConditionEncoder(input_dim, lookback, condition_dim)
        self.time_embedding = nn.Embedding(num_timesteps, hidden_dim)
        self.model = nn.Sequential(
            nn.Linear(1 + condition_dim + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x_condition, y_noisy, t):
        cond = self.condition_encoder(x_condition)
        t_emb = self.time_embedding(t)
        inp = torch.cat([y_noisy, cond, t_emb], dim=1)
        predicted_noise = self.model(inp)
        return predicted_noise
    
    def sample(self, x_condition, device):
        batch_size = x_condition.size(0)
        y = torch.randn(batch_size, 1, device=device)
        for t in reversed(range(self.num_timesteps)):
            t_tensor = torch.full((batch_size,), t, device=device, dtype=torch.long)
            predicted_noise = self.forward(x_condition, y, t_tensor)
            alpha = self.alphas[t]
            alpha_cumprod = self.alphas_cumprod[t]
            beta = self.betas[t]
            y = (1 / torch.sqrt(alpha)) * (y - (beta / torch.sqrt(1 - alpha_cumprod)) * predicted_noise)
            if t > 0:
                noise = torch.randn_like(y)
                y = y + torch.sqrt(beta) * noise
        return y

# --- 모델 로드 (동일) ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = 400
lookback = 6
model = DiffusionClassifier(input_dim=input_dim, lookback=lookback, condition_dim=128, num_timesteps=100, hidden_dim=128).to(device)
model_path = "diffusion_model_experiment_14_6.pth"  # 실제 모델 파일 경로로 수정
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# --- Bybit 거래 심볼 설정 ---
symbol = "ETHUSDT"  # USDT 페어

# --- 예측 함수 ---
def get_model_prediction():
    df = get_latest_kline(symbol, interval="5", limit=15, category="linear")
    if df is None or len(df) < lookback:
        notify("예측에 충분한 데이터가 없습니다.")
        return None
    ohlc_features = ['open', 'high', 'low', 'close']
    for feature in ohlc_features:
        df[feature] = rolling_minmax_scale(df[feature], window=6)
    df_processed = bin_and_encode(df.copy(), ohlc_features, bins=100, drop_original=True)
    final_input_columns = [col for col in df_processed.columns if '_Bin_' in col]
    if len(df_processed) < lookback:
        notify("lookback 데이터가 부족합니다.")
        return None
    input_seq = df_processed[final_input_columns].iloc[-lookback:]
    x = torch.tensor(input_seq.values, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        y_cont = model.sample(x, device)
        notify("디버그: y_cont = " + str(y_cont.item()))
        prediction = 1 if y_cont.item() >= 0.5 else 0
    return prediction

# --- 글로벌 변수: pending_trade와 거래 통계 ---
pending_trade = None
stats = {
    'total_trades': 0,
    'win_count': 0,
    'cumulative_return': 1.0,
    'trade_returns': []
}

# --- 거래 실행 함수 (신규 거래 진입) ---
def trade_decision():
    global pending_trade
    prediction = get_model_prediction()
    if prediction is None:
        return
    entry_price = get_current_price(symbol)
    if entry_price is None:
        notify("현재가를 조회하지 못했습니다.")
        return
    if prediction == 1:
        usdt_balance = get_balance_usdt()
        print(usdt_balance)
        if usdt_balance is None or usdt_balance < 5:
            notify("매수할 충분한 USDT 잔고가 없습니다.")
            return
        trade_amount_usdt = usdt_balance * 0.5
        coins_bought = trade_amount_usdt / entry_price
        notify(f"[진입] 상승 예측: USDT 잔고의 50%({trade_amount_usdt:.2f} USDT)로 매수, 진입가: {entry_price:.2f}, 구매 개수: {coins_bought:.4f} 개")
        place_order(symbol, side="Buy", qty=coins_bought, category="linear")
        pending_trade = {
            'direction': prediction,
            'entry_price': entry_price,
            'quantity': coins_bought,
            'entry_time': pd.Timestamp.now()
        }
    else:
        usdt_balance = get_balance_usdt()
        print(usdt_balance)
        if usdt_balance is None or usdt_balance < 5:
            notify("숏 포지션 진입할 충분한 USDT 잔고가 없습니다.")
            return
        trade_amount_usdt = usdt_balance * 0.5
        coins_sold = trade_amount_usdt / entry_price
        notify(f"[진입] 하락 예측: USDT 잔고의 50%({trade_amount_usdt:.2f} USDT)로 숏 포지션 진입, 진입가: {entry_price:.2f}, 주문 수량: {coins_sold:.4f} 개")
        place_order(symbol, side="Sell", qty=coins_sold, category="linear")
        pending_trade = {
            'direction': prediction,
            'entry_price': entry_price,
            'quantity': coins_sold,
            'entry_time': pd.Timestamp.now()
        }

# --- 거래 결과 처리 함수 (청산 시점에서 거래 결과 계산) ---
def process_pending_trade():
    global pending_trade, stats
    if pending_trade is None:
        return
    exit_price = get_current_price(symbol)
    if exit_price is None:
        notify("청산 가격을 조회하지 못했습니다.")
        return
    direction = pending_trade['direction']
    entry_price = pending_trade['entry_price']
    fee = 0.0005  # 0.05%
    if direction == 1:
        fraction = 0.5
        effective_factor = (exit_price * (1 - fee)) / (entry_price * (1 + fee))
        trade_return = 1 + fraction * (effective_factor - 1)
        trade_type = "Long"
        outcome = "Hit" if exit_price > entry_price else "Miss"
    else:
        fraction = 0.5
        effective_factor = (entry_price * (1 - fee)) / (exit_price * (1 + fee))
        trade_return = 1 + fraction * (effective_factor - 1)
        trade_type = "Short"
        outcome = "Hit" if exit_price <= entry_price else "Miss"
    
    stats['total_trades'] += 1
    if outcome == "Hit":
        stats['win_count'] += 1
    stats['cumulative_return'] *= trade_return
    stats['trade_returns'].append(trade_return)
    quantity = pending_trade.get('quantity', 0)
    notify(f"[청산] {trade_type} 거래 - 진입가: {entry_price:.2f}, 청산가: {exit_price:.2f}, 거래 개수: {quantity:.4f} 개, 거래 수익률: {trade_return:.4f} ({outcome})")
    hit_ratio = stats['win_count'] / stats['total_trades'] if stats['total_trades'] > 0 else 0
    notify(f"거래 횟수: {stats['total_trades']}, Hit Ratio: {hit_ratio:.2%}, 누적 수익률: {stats['cumulative_return']:.4f}\n")
    pending_trade = None

# --- 메인 루프: 새로운 5분봉이 생성될 때마다 업데이트 ---
last_candle_time = None
while True:
    try:
        df = get_latest_kline(symbol, interval="5", limit=1, category="linear")
        if df is not None and not df.empty:
            current_candle_time = df.index[-1]
            if last_candle_time is None or current_candle_time > last_candle_time:
                if pending_trade is not None:
                    process_pending_trade()
                notify(f"새로운 5분봉 생성: {current_candle_time}")
                trade_decision()
                last_candle_time = current_candle_time
        else:
            notify("최신 5분봉 데이터를 불러오지 못했습니다.")
    except Exception as e:
        notify(f"에러 발생: {e}")
    time.sleep(10)


새로운 5분봉 생성: 2025-03-16 09:40:00
디버그: y_cont = 1.0379494428634644
⚠ 응답 내용이 비어 있습니다 (get_current_price) - 재시도 1/3
⚠ 응답 내용이 비어 있습니다 (get_current_price) - 재시도 2/3
⚠ 응답 내용이 비어 있습니다 (get_current_price) - 재시도 3/3
🚨 get_current_price() 요청 실패: 모든 재시도 실패
현재가를 조회하지 못했습니다.


KeyboardInterrupt: 